In [2]:
##2b_Qapp_PrimaryCareDELTAtoStaging.ipynb - last modified 20230302 
## 20230302 - Added in srorganisation
## 20230208 - to account for thosepatients with no DOB etc as they are skeewing the data - they will be removed from the local persons table
## 20230209 - to account for persons not in obs period. Notes below 
#Builds the PrimaryCare FDM
#Assumes that the targetdb below exists
#Needs code to build it if it doesn't

#You will need to manually amend the target dataset for this script

library(bigrquery)
library(lubridate) # required for datetime

# Store the project ID
project_id = "yhcr-prd-phm-bia-core"

sourcedb <-'yhcr-prd-phm-bia-core.CB_MYSPACE_JDB'
targetdb <-'yhcr-prd-phm-bia-core.CB_STAGING_DATABASE_PrimaryCare'
    demo <-'yhcr-prd-phm-bia-core.CB_STAGING_DATABASE.src_DemoGraphics_MASTER'
targetdb <-gsub(' ','',targetdb)
print (sourcedb) 
print (targetdb)
print (demo)

[1] "yhcr-prd-phm-bia-core.CB_MYSPACE_JDB"
[1] "yhcr-prd-phm-bia-core.CB_STAGING_DATABASE_PrimaryCare"
[1] "yhcr-prd-phm-bia-core.CB_STAGING_DATABASE.src_DemoGraphics_MASTER"


In [ ]:
# Sample script
sql1 <-paste('drop table if exists ',targetdb,'.care_site', sep = "")
tb1 <- bq_project_query(project_id, sql1)

In [ ]:

#tbl_SRAppointment

sql1 <-paste('delete from ', sourcedb,'.tbl_SRAppointment` 
where digest is null ', sep = "")
tb1 <- bq_project_query(project_id, sql1)

sql2 <-paste('delete from ', sourcedb,'. tbl_SRAppointment` 
where RowIdentifier like \'%---%\'; ', sep = "")
tb2 <- bq_project_query(project_id, sql2)

sql3 <-paste('insert into  ',targetdb,'.tbl_SRAppointment 
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
demo.person_id ,
IDOrganisationVisibleTo,
cast( DateStart as Datetime) as DateStart ,
cast(DateEnd as Datetime) as  DateEnd,
AppointmentStatus,
DateAppointmentBooked,
DatePatientArrival,
DatePatientSeen,
FollowUpAppointment,
TelephoneAppointment,
IDClinician,
IDProfileClinician,
RotaName,
RotaType,
RotaLocation,
RotaCode,
IDRotaOwner,
IDProfileRotaOwner,
AllowsOverbooking,
BookingContactNumber,
DateAppointmentCancelled,
IDRota,
IDReferralIN,
IDPatient,
apt.Digest,
IDOrganisation,
IDOrganisationRegisteredAt,
DataDeleted,
LoadNum,
deleteflag,
LoadedToFDM ,
Cast(cb_rowid as int64) 
From ', sourcedb,'tbl_SRAppointment` apt
left join ', demo,' demo
on apt.Digest = demo.digest; ', sep = "")

tb3 <- bq_project_query(project_id, sql3)


eric <- paste("tbl_SRAppointment done "  ,Sys.time() + hours(1) ,sep = "")
print(eric) 


In [ ]:
#tbl_SRCHStatusHistory


sql4 <-paste('delete from ', sourcedb,'tbl_SRCHSStatusHistory where Digest is null', sep = "")
tb4 <- bq_project_query(project_id, sql4)

sql5 <-paste('insert into   ',targetdb,'.tbl_SRCHSStatusHistory`
SELECT cast(RowIdentifier as int64) as RowIdentifier, 
IDOrganisationVisibleTo, 
IDEvent, 
IDPatient, 
Digest, 
Cast(left(DateEventRecorded,19) as date) as DateEventRecorded,
Cast(left(DateEvent,19)  as date) as DateEvent,
IDProfileEnteredBy, 
IDDoneBy, 
IDOrganisationDoneAt, 
TextualEventDoneBy, 
CHSStatusGroup, 
CHSStatus, 
IDOrganisation, 
IDOrganisationRegisteredAt, 
DataDeleted, 
LoadNum, 
deleteflag, 
LoadedToFDM  ,
cast(cb_rowid as int64) 
From ', sourcedb,'.tbl_SRCHSStatusHistory', sep = "")

tb5 <- bq_project_query(project_id, sql5)

eric <- paste("tbl_SRCHStatusHistory done "  ,Sys.time() + hours(1) ,sep = "")
print(eric) 

In [ ]:
sql1 <-paste(' ', sep = "")
tb1 <- bq_project_query(project_id, sql1)

In [ ]:
sql1 <-paste(' ', sep = "")
tb1 <- bq_project_query(project_id, sql1)

In [ ]:


 

delete from ', sourcedb,'.tbl_SRCode
where Digest is null;

insert into   ',targetdb,'.tbl_SRCode
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
demo.person_id ,
IDOrganisationVisibleTo,
Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
Cast(left(DateEvent,19)  as datetime) as DateEvent,
cast(IDProfileEnteredBy as int64) as IDProfileEnteredBy,
cast(IDDoneBy as int64) as IDDoneBy ,
TextualEventDoneBy,
IDOrganisationDoneAt,
CTV3Code,
CTV3Text,
SNOMEDCode,
SNOMEDText,
cast(NumericComparator as int64) ,
IsNumeric,
cast(NumericValue as float64) ,
NumericUnit,
cast(EpisodeType as int64),
cast(IDTemplate as int64),
cast(IDEvent as int64),
cast(IDPatient as int64),
cod.Digest,
cast(IDReferralIN as int64),
cast(IDAppointment as int64),
cast(IDVisit as int64),
IDOrganisation,
IDOrganisationRegisteredAt,
DataDeleted,
deleteflag,
'0'as LoadedToGCP,
LoadNum,
LoadedToFDM ,
cast(cb_rowid as int64) 
From ', sourcedb,'.tbl_SRCode cod
left join ', demo, '  demo
on cod.Digest = demo.digest ;


update  ',targetdb,'.tbl_SRCode src
set src.SNOMEDCode = sno.SNOMEDCode
 from yhcr-prd-phm-bia-core.CB_LOOKUPS.tbl_CTV3ToSnomed_Mapsno
 where src.CTV3Code = sno.CTV3Code
 and src.SNOMEDCode is null;
 
#tbl_SRCodeTemplateLink

delete from ', sourcedb,'.tbl_SRCodeTemplateLink 
where Digest is null;

insert into   ',targetdb,'.tbl_SRCodeTemplateLink
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
demo.person_id ,
IDPatient,
IDOrganisationVisibleTo,
IDEvent,
IDCode,
IDTemplateRecording,
IDTemplate,
DateEventRecorded,
cod.digest,
DataDeleted,
LoadNum,
deleteflag,
LoadedToFDM ,
cast(cb_rowid as int64) 
From ', sourcedb,'.tbl_SRCodeTemplateLink cod
left join ', demo, '  demo
on cod.Digest = demo.digest;
#tbl_SRConfiguredListOption

delete From ', sourcedb,'.tbl_SRConfiguredListOption
where ConfiguredListOption is null;

delete From ', sourcedb,'.tbl_SRConfiguredListOption
where RowIdentifier like '%---%';

insert into  ',targetdb,'.tbl_SRConfiguredListOption 
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier,
 ConfiguredList,
ConfiguredListOption,
CDSCode,
CAMHSCode,
MHSDSCode,
DataDeleted,
LoadNum,
deleteflag,
LoadedToFDM ,
cast(cb_rowid as int64) as cb_rowid
 From ', sourcedb,'.tbl_SRConfiguredListOption ;
 
#tbl_DrugSensitivity
delete from ', sourcedb,'.tbl_SRDrugSensitivity
where Digest is null;
delete from ', sourcedb,'.tbl_SRDrugSensitivity
where IDOrganisationVisibleTo is null;
#delete from  ',targetdb,'.tbl_SRDrugSensitivity
#where RowIdentifier in (SELECT distinct cast(RowIdentifier as int64)  from   ', sourcedb,'.tbl_SRDrugSensitivity);
insert into  ',targetdb,'.tbl_SRDrugSensitivity
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
demo.person_id ,
IDOrganisationVisibleTo,
Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
Cast(left(DateEvent,19)  as datetime) as DateEvent,
IDProfileEnteredBy,
IDDoneBy,
TextualEventDoneBy,
IDOrganisationDoneAt,
Cast(left(DateStarted,19) as datetime) as DateStarted,
Cast(left(DateEnded,19) as datetime) as DateEnded,
FormulationSpecific,
IDDrugCode,
IDMultiLexAction,
IDReferralIn,
IDEvent,
IDPatient,
drg.Digest,
IDOrganisation,
IDOrganisationRegisteredAt,
DataDeleted,
LoadNum,
deleteflag,
LoadedToFDM,
cast(cb_rowid as int64) 
 From ', sourcedb,'.tbl_SRDrugSensitivitydrg
left join ', demo, '  demo
on drg.Digest = demo.digest;

#tbl_SREvent
delete from ', sourcedb,'.tbl_SREvent
where Digest is null;
delete from ', sourcedb,'.tbl_SREvent
where IDOrganisationVisibleTo is null;
#delete from  ',targetdb,'.tbl_SREvent
#where RowIdentifier in (SELECT distinct cast(RowIdentifier as int64)  from   ', sourcedb,'.tbl_SREvent);
insert into  ',targetdb,'.tbl_SREvent
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
demo.person_id ,
IDOrganisationVisibleTo,
Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
Cast(left(DateEvent,19)  as datetime) as DateEvent,
IDProfileEnteredBy,
IDDoneBy,
IDStaffMemberProfileRole,
TextualEventDoneBy,
IDAuthorisedBy,
IDProfileAuthorisedBy,
ContactEventLocation,
ContactMethod,
EventIncomplete,
ClinicalEvent,
IDReferralin,
IDPatient,
Sre.Digest,
IDOrganisation,
IDTeam,
IDBranch,
IDOrganisationRegisteredAt,
DataDeleted,
IDOrganisationDoneAt,
deleteflag,
LoadedToFDM,
LoadNum ,
cast(cb_rowid as int64) 
From ', sourcedb,'.tbl_SREvent sre 
left join ', demo, '  demo
on sre.Digest = demo.digest;
#tbl_SREventLink
delete from ', sourcedb,'.tbl_SREventLink
where Digest is null;
delete from ', sourcedb,'.tbl_SREventLink
where IDOrganisationVisibleTo is null;
#delete from  ',targetdb,'.tbl_SREventLink
#where RowIdentifier in (SELECT distinct cast(RowIdentifier as int64)  from   ', sourcedb,'.tbl_SREventLink);
insert into  ',targetdb,'.tbl_SREventLink 
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
demo.person_id ,
 IDOrganisationVisibleTo,
Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
Cast(left(DateEvent,19)  as datetime) as DateEvent
,IDProfileEnteredBy
,IDDoneBy
,TextualEventDoneBy
,IDOrganisationDoneAt
,IDEvent
,IDPatient
,sre.Digest
,IDReferralin
,IDOrganisation
,IDAppointment
,IDVisit
,IDOrganisationRegisteredAt
,DataDeleted
,LoadNum
,deleteflag
,LoadedToFDM ,
cast(cb_rowid as int64) 
From ', sourcedb,'.tbl_SREventLink sre
left join ', demo, '  demo
on sre.Digest = demo.digest;
#tbl_SRGPPracticeHistory
delete from ', sourcedb,'.tbl_SRGPPracticeHistory
where Digest is null;
delete from ', sourcedb,'.tbl_SRGPPracticeHistory
where IDOrganisationVisibleTo is null;
#delete from  ',targetdb,'.tbl_SRGPPracticeHistory
#where RowIdentifier in (SELECT distinct cast(RowIdentifier as int64)  from   ', sourcedb,'.tbl_SRGPPracticeHistory);
insert into  ',targetdb,'.tbl_SRGPPracticeHistory 
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier,
 IDOrganisationVisibleTo,
  IDPractice,
  IDProfileRegisteredGP,
  cast( DateFrom as Datetime) as DateFrom ,
cast(DateTo as Datetime) as  DateTo,
     IDPatient,
     prac.Digest,
     DataDeleted,
      deleteflag,
      LoadedToFDM ,
cast(cb_rowid as int64) 
      From ', sourcedb,'.tbl_SRGPPracticeHistory prac
      left join ', demo, '  demo
on prac.Digest = demo.digest;
 
#tbl_SRGoal
 
delete from ', sourcedb,'.tbl_SRGoal
where Digest is null ;
delete from ', sourcedb,'.tbl_SRGoal
where IDOrganisationVisibleTo is null;

insert into  ',targetdb,'.tbl_SRGoal 
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
demo.person_id ,
 IDOrganisationVisibleTo,
Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
Cast(left(DateEvent,19)  as datetime) as DateEvent,
IDProfileEnteredBy,
IDDoneBy,
TextualEventDoneBy,
IDOrganisationDoneAt,
GoalCategory,
GoalType,
GoalOutcome,
GoalImportance,
DateGoalTarget,
ActionPlanCategory,
ActionPlanPatientConfidence,
ActionPlanOutcome,
DateEndedActionPlan,
IDEvent,
IDPatient,
IDOrganisation,
IDOrganisationRegisteredAt,
GoalPerformance,
Gol.digest,
DataDeleted,
LoadNum,
deleteflag,
LoadedToFDM ,
cast(cb_rowid as int64) 
From ', sourcedb,'.tbl_SRGoal gol
      left join ', demo, '  demo
on gol.Digest = demo.digest;

#tbl_SRImmunisation
delete from ', sourcedb,'.tbl_SRImmunisation
where Digest is null;
delete from ', sourcedb,'.tbl_SRImmunisation
where IDOrganisationVisibleTo is null;

insert into  ',targetdb,'.tbl_SRImmunisation 
SELECT distinct  cast(RowIdentifier as int64) as RowIdentifier ,
demo.person_id ,
 IDOrganisationVisibleTo,
Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
Cast(left(DateEvent,19)  as datetime) as DateEvent,
IDProfileEnteredBy,
IDDoneBy,
TextualEventDoneBy,
IDOrganisationDoneAt,
IDVaccination,
IDImmunisationContent,
Dose,
Location,
Method,
DateExpiry,
ImmsReadCode,
VaccPart,
VaccBatchNumber,
VaccAreaCode,
VaccinationStatus,
IDEvent,
IDPatient,
imm.Digest,
IDOrganisation,
IDOrganisationRegisteredAt,
DataDeleted,
deleteflag,
LoadNum,
LoadedToFDM ,
cast(cb_rowid as int64) 
From ', sourcedb,'.tbl_SRImmunisationimm
  left join ', demo, '  demo
on imm.Digest = demo.digest;
 
#tbl_SRImmunisationConsent
 
delete from ', sourcedb,'.tbl_SRImmunisationConsent
where Digest is null;
delete from ', sourcedb,'.tbl_SRImmunisationConsent
where IDOrganisationVisibleTo is null;

insert into  ',targetdb,'.tbl_SRImmunisationConsent 
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
demo.person_id ,
 IDOrganisationVisibleTo,
Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
Cast(left(DateEvent,19)  as datetime) as DateEvent,
IDProfileEnteredBy,
IDDoneBy,
TextualEventDoneBy,
IDOrganisationDoneAt,
ConsentedToVacc,
RefusalOrConsentCurrent,
DateEnded,
VaccName,
VaccContent,
IDEvent,
IDPatient,
Imm.Digest,
IDOrganisation,
IDOrganisationRegisteredAt,
DataDeleted,
LoadNum,
deleteflag,
LoadedToFDM ,
cast(cb_rowid as int64) 
From ', sourcedb,'.tbl_SRImmunisationConsent imm
  left join ', demo, '  demo
on imm.Digest = demo.digest;
 
#tbl_SRImmunisationContent
delete from   ', sourcedb,'.tbl_SRImmunisationContent
where Name is null ;
insert into  ',targetdb,'.tbl_SRImmunisationContent 
SELECT distinct cast(RowIdentifier as int64) 
, Name
, Content
, DateDeleted
, DataDeleted
, cast(LoadNum as int64) 
, deleteflag
, LoadedToFDM ,
cast(cb_rowid as int64) 
From ', sourcedb,'.tbl_SRImmunisationContent ;

#tbl_Mapping
delete from   ', sourcedb,'.tbl_SRMapping
where Mapping is null ;

insert into  ',targetdb,'.tbl_SRMapping 
SELECT
cast(RowIdentifier as int64) as RowIdentifier
, IdMappingGroup
, Mapping
, DataDeleted
, LoadNum
, deleteflag
, LoadedToFDM,
cast(cb_rowid as int64) 
 From ', sourcedb,'.tbl_SRMapping;
#tbl_MappingGroup
delete from   ', sourcedb,'.tbl_SRMappingGroup
where GroupName is null ;

insert into  ',targetdb,'.tbl_SRMappingGroup 
SELECT
cast(RowIdentifier as int64) as RowIdentifier
, GroupName
, DataDeleted
, LoadNum
, deleteflag
, LoadedToFDM,
cast(cb_rowid as int64) 
 From ', sourcedb,'.tbl_SRMappingGroup;
#tbl_SRMedicationReadCodeDetails 
delete from   ', sourcedb,'.tbl_SRMedicationReadCodeDetails
where IDMultiLexProduct is null ;
#delete from  ',targetdb,'.tbl_SRMedicationReadCodeDetails
#where RowIdentifier in (SELECT distinct cast(RowIdentifier as int64)  from   ', sourcedb,'.tbl_SRMedicationReadCodeDetails);
insert into  ',targetdb,'.tbl_SRMedicationReadCodeDetails 
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier 
, IDMultiLexProduct
, DrugReadCode
, DrugReadCodeDesc
, DataDeleted
, LoadNum
, deleteflag
, LoadedToFDM ,
cast(cb_rowid as int64) 
From ', sourcedb,'.tbl_SRMedicationReadCodeDetails ;
 
 

#tbl_SROnlineServices
delete from   ', sourcedb,'.tbl_SROnlineServices 
where Digest is null ;

#delete from  ',targetdb,'.tbl_SROnlineServices 
#where RowIdentifier in (SELECT distinct cast(RowIdentifier as int64)  from   ', sourcedb,'.tbl_SROnlineServices);
insert into  ',targetdb,'.tbl_SROnlineServices
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
 IDEvent, 
 IDPatient,
 Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded
, cast(left(DateEvent,19) as datetime) as DateEvent
, IDProfileEnteredBy, 
IDDoneBy,
 TextualEventDoneBy, 
 IDOrganisation, 
 IDOrganisationDoneAt, 
 ViewRecord, 
 Appointments, 
 Prescriptions, 
 Questionnaires, 
 SummaryCareRecord, 
 DateDeleted, 
 IDProfileDeletedBy, 
 DetailedCodedRecord, 
 IDOnlineUser, 
 IDOrganisationRegisteredAt, 
 sv.digest, 
 DataDeleted, 
 LoadNum, 
 deleteflag,
LoadedToFDM,
cast(cb_rowid as int64) 
From ', sourcedb,'.tbl_SROnlineServices sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;
#tbl_SROrganisation 
delete from   ', sourcedb,'.tbl_SROrganisation where Name is null ;

insert into  ',targetdb,'.tbl_SROrganisation
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
Name, 
 ID, 
 HouseName, 
 HouseNumber, 
 NameOfRoad, 
 NameOfLocality, 
 NameOfTown,
  NameOfCounty, 
  FullPostCode, 
  Telephone, 
  SecondaryTelephone,
   Fax, 
  MadeObsolete, 
  IDTrust, 
  IDCcg, 
  DataDeleted

From ', sourcedb,'.tbl_SROrganisation  sv;
 
#tbl_SROrganisationBranch
delete from   ', sourcedb,'.tbl_SROrganisationBranch where BranchName is null ;

insert into  ',targetdb,'.tbl_SROrganisationBranch
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier, 
IDOrganisationVisibleTo, 
ID, 
BranchName, 
HouseName, 
HouseNumber, 
RoadName, 
Locality, 
Town, 
County, 
PostCode, 
BranchObsolete, 
IDOrganisation, 
DataDeleted, 
LoadNum, 
deleteflag, 
LoadedToFDM ,
cast(cb_rowid as int64) 
From ', sourcedb,'.tbl_SROrganisationBranch ;
 
#tbl_SROverseasVisitorChargingCategory
delete from   ', sourcedb,'.tbl_SROverseasVisitorChargingCategory where Digest is null ;


insert into  ',targetdb,'.tbl_SROverseasVisitorChargingCategory
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
 IDEvent, 
 OverseasVisitorChargingCategory, 
 Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
 Cast(left(DateEvent,19) as datetime) as DateEvent,
IDProfileEnteredBy, 
IDDoneBy, 
TextualEventDoneBy, 
IDOrganisationDoneAt, 
IDOrganisation, 
IDPatient, 
IDReferralIn, 
DataDeleted, 
sv.digest, 
LoadNum, 
deleteflag, 
LoadedToFDM ,
cb_rowid 
From ', sourcedb,'.tbl_SROverseasVisitorChargingCategory sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;
#tbl_SRPatient
delete from   ', sourcedb,'.tbl_SRPatient where Digest is null ;

insert into  ',targetdb,'.tbl_SRPatient
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
 sv.Digest, 
 DateBirth, 
 DateDeath, 
 BirthPlace, 
 Gender, 
 SpeaksEnglish, 
 TestPatient, 
 SSRef, 
 SpineMatched, 
 Ethnicity, 
 LanguageSpoken, 
 SOA, 
 sv.Ward, 
 Religion, 
 DataDeleted, 
 LoadNum, 
 deleteflag, 
 LoadedToFDM ,
cb_rowid 
 From ', sourcedb,'.tbl_SRPatient  sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;
#tbl_SRPatientAddressHistory
delete from   ', sourcedb,'.tbl_SRPatientAddressHistory where Digest is null ;

insert into  ',targetdb,'.tbl_SRPatientAddressHistory
SELECT distinct cast(RowIdentifier as int64) ,
person_id,
IDOrganisationVisibleTo,
 Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
IDProfileEnteredBy,
IDDoneBy,
TextualEventDoneBy,
IDOrganisationDoneAt,
sv.PartialPostCode,
DateTo,
AddressType,
IDEvent,
IDPatient,
sv.Digest,
IDOrganisation,
CcgOfResidence,
IDOrganisationRegisteredAt,
DataDeleted,
LoadNum,
deleteflag,
LoadedToFDM ,
cb_rowid 
From ', sourcedb,'.tbl_SRPatientAddressHistory sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;
#tbl_SRPatientGroups
delete from   ', sourcedb,'.tbl_SRPatientGroups where Digest is null ;

insert into  ',targetdb,'.tbl_SRPatientGroups
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
 Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
 Cast(left(DateEvent,19) as datetime) as DateEvent,
 IDProfileEnteredBy, 
Name, 
GroupType, 
Members, 
DateRemoved, 
IDStaffRemoved, 
IDProfileRemoved, 
IDEvent, 
IDPatient, 
IDOrganisation, 
IDOrganisationRegisteredAt, 
sv.digest, 
DataDeleted, 
LoadNum, 
deleteflag, 
LoadedToFDM ,
cb_rowid 
 From ', sourcedb,'.tbl_SRPatientGroups sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;

#tbl_SRPatientRegistration
delete from   ', sourcedb,'.tbl_SRPatientRegistration where Digest is null ;

insert into  ',targetdb,'.tbl_SRPatientRegistration
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
 Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
 #Cast(left(DateEvent,19) as datetime) as DateEvent,
IDProfileEnteredBy, 
DateRegistration, 
DateDeRegistration, 
RegistrationStatus, 
PreferredPharmacy,
 IDPatient, 
 sv.Digest, 
 IDOrganisation, 
 IDOrganisationRegisteredAt, 
 DataDeleted, 
 LoadNum, 
 deleteflag, 
 LoadedToFDM ,
cb_rowid 
 From ', sourcedb,'.tbl_SRPatientRegistration  sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;

#tbl_SRPatientRelationship
delete from   ', sourcedb,'.tbl_SRPatientRelationship where Digest is null ;

insert into  ',targetdb,'.tbl_SRPatientRelationship
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
 Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
 Cast(left(DateEvent,19) as datetime) as DateEvent,
IDProfileEnteredBy,
IDDoneBy,
TextualEventDoneBy,
IDOrganisationDoneAt,
DateEnded,
RelationshipType,
PersonalGuardianOrProxy,
NextOfKin,
CaresForPatient,
PrincipalCarerForPatient,
KeyHolder,
HasParentalResponsibility,
FinancialRepresentative,
Advocate,
MainVisitor,
CallCentreCallBackConsent,
CopyCorrespondence,
ContactOrder,
ContactMethod,
CommunicationFormat,
InterpreterRequired,
IDRelationshipWithPatient,
IDPatientRelationshipWith,
CodeRelationshipWithUser,
RelationshipWithOrganisation,
IDEvent,
IDPatient,
IDOrganisation,
IDOrganisationRegisteredAt,
sv.digest,
DataDeleted,
LoadNum,
deleteflag,
LoadedToFDM 
,cast(cb_rowid as int64)  
From ', sourcedb,'.tbl_SRPatientRelationship sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;



#THIS NEEDS CHECKING !!!! AS MISSING LOADS OF FIELDS !!

#tbl_SRPersonAtRisk
delete from   ', sourcedb,'.tbl_SRPersonAtRisk where Digest is null ;

insert into  ',targetdb,'.tbl_SRPersonAtRisk
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
DateAdded,
IDProfileEnteredBy,
ProtectionPlan,
DateRemoved,
IDProfileRemovedBy,
IDPatient,
IDOrganisation,
IDOrganisationRegisteredAt,
sv.digest,
DataDeleted,
LoadNum,
deleteflag,
LoadedToFDM ,
cast(cb_rowid as int64) 
From ', sourcedb,'.tbl_SRPersonAtRisk sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;






#tbl_SRPrimaryCareMedication 
delete from   ', sourcedb,'.tbl_SRPrimaryCareMedication 
where Digest is null ;

insert into  ',targetdb,'.tbl_SRPrimaryCareMedication 
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
Cast(left(DateEvent,19)  as datetime) as DateEvent,
 IDProfileEnteredBy,
 IDDoneBy,
 TextualEventDoneBy,
 IDOrganisationDoneAt,
 IDMultiLexProduct,
 IDMultiLexPack,
 IDMultiLexDMD,
 NameOfMedication,
Cast(left(DateMedicationStart,19) as datetime) as DateMedicationStart,
Cast(left(DateMedicationEnd,19)  as datetime) as DateMedicationEnd,
 MedicationDosage,
 MedicationQuantity,
 IsSupervised,
 SupervisedDose,
 UnSupervisedDose,
 IsRepeatMedication,
 IsOtherMedication,
 IsDentalMedication,
 IsHospitalMedication,
 IDRepeatTemplate,
 IDReferralIN,
 IDEvent,
 IDPatient,
 med.Digest,
 IDOrganisation,
 IDOrganisationRegisteredAt,
 DataDeleted,
 LoadNum,
 deleteflag,
 LoadedToFDM ,
cast(cb_rowid as int64) 
 From ', sourcedb,'.tbl_SRPrimaryCareMedication med 
  left join ', demo, '  demo
on med.Digest = demo.digest;

#tbl_SRProblem
delete from   ', sourcedb,'.tbl_SRProblem where Digest is null ;

insert into  ',targetdb,'.tbl_SRProblem
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
 Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
 Cast(left(DateEvent,19) as datetime) as DateEvent,
IDProfileEnteredBy,
IDDoneBy,
TextualEventDoneBy,
IDOrganisationDoneAt,
CTV3Code,
DateEnd,
Severity,
IDCode,
IDEvent,
IDPatient,
sv.Digest,
IDReferralIN,
IDOrganisation,
IDOrganisationRegisteredAt,
DataDeleted,
LoadNum,
deleteflag,
LoadedToFDM ,
cast(cb_rowid as int64) 
From ', sourcedb,'.tbl_SRProblem  sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;

#tbl_SRRecall
delete from   ', sourcedb,'.tbl_SRRecall where Digest is null ;

insert into  ',targetdb,'.tbl_SRRecall
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
 Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
 Cast(left(DateEvent,19) as datetime) as DateEvent,
  IDProfileEnteredBy,
IDDoneBy,
TextualEventDoneBy,
IDOrganisationDoneAt,
DateRecall,
RecallType,
RecallInitialStatus,
RecallStatus,
RecallStatusDate,
IDReferralIN,
IDEvent,
IDPatient,
sv.Digest,
IDOrganisation,
IDOrganisationRegisteredAt,
DataDeleted,
LoadNum,
deleteflag,
LoadedToFDM ,
cast(cb_rowid as int64) 
From ', sourcedb,'.tbl_SRRecallsv
  left join ', demo, '  demo
on sv.Digest = demo.digest;




#tbl_SRReferralIn
delete from   ', sourcedb,'.tbl_SRReferralIn where Digest is null ;

insert into  ',targetdb,'.tbl_SRReferralIn
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
 Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
 Cast(left(DateEvent,19) as datetime) as DateEvent,
 IDProfileEnteredBy,
IDDoneBy,
TextualEventDoneBy,
IDOrganisationDoneAt,
Category,
PrimaryReason,
DateReferral,
DateOnReferralLetter,
Source,
ServiceOffered,
ServiceType,
ReReferral,
Urgency,
IDCodePrimaryDiagnosis,
ReferralCode,
IDReferralLocal,
ReasonForServiceDelay,
Outcome,
DefaultContactLocation,
DateDischargeScheduled,
DateDischarge,
DischargeReason,
DischargeLocation,
IDReferrer,
SourceTableReferrer,
PatientAware,
DateOfAction,
ActiveStatus,
ReferredToSpeciality,
ReferredToTreatmentCode,
IDReferredTo,
SourceTableReferrerTo,
ReferralPriority,
ReferralMedium,
DateOfDecision,
DayCareReferral,
MaternityReferral,
Commissioner,
ClientCategory,
PrimarySupportReason,
IDCaseload,
IDEvent,
IDPatient,
sv.Digest,
IDOrganisation,
ReferralEndReason,
HospitalReferralSourceCdsCode,
M101140ReasonForOutOfAreaReferralAdultAcuteMentalHealth,
IDOrganisationRegisteredAt,
HospitalReferredForType,
DataDeleted,
LoadNum,
deleteflag,
LoadedToFDM ,
cb_rowid 
From ', sourcedb,'.tbl_SRReferralIn sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;









#tbl_SRReferralInIntervention
delete from   ', sourcedb,'.tbl_SRReferralInIntervention where Digest is null ;


insert into  ',targetdb,'.tbl_SRReferralInIntervention
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
 Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
 Cast(left(DateEvent,19) as datetime) as DateEvent,
  IDProfileEnteredBy,
IDDoneBy,
TextualEventDoneBy,
IDOrganisationDoneAt,
Intervention,
IDReferralIN,
IDEvent,
IDPatient,
sv.Digest,
IDOrganisation,
IDOrganisationRegisteredAt,
DataDeleted,
LoadNum,
deleteflag,
LoadedToFDM ,
cast(cb_rowid as int64) 
 From ', sourcedb,'.tbl_SRReferralInIntervention sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;






#tbl_SRReferralInReferralReason
delete from   ', sourcedb,'.tbl_SRReferralInReferralReason where Digest is null ;

insert into  ',targetdb,'.tbl_SRReferralInReferralReason
 SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
 Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
 Cast(left(DateEvent,19) as datetime) as DateEvent,
IDProfileEnteredBy,
IDDoneBy,
TextualEventDoneBy,
IDOrganisationDoneAt,
PrimaryReferralReason,
ReferralReason,
DateRemoved,
IDReferralIN,
IDEvent,
IDPatient,
sv.Digest,
IDOrganisation,
IDOrganisationRegisteredAt,
DataDeleted,
LoadNum,
deleteflag,
LoadedToFDM ,
cast(cb_rowid as int64) 
From ', sourcedb,'.tbl_SRReferralInReferralReason sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;









#tbl_SRReferralInStatusDetails
delete from   ', sourcedb,'.tbl_SRReferralInStatusDetails where Digest is null ;

insert into  ',targetdb,'.tbl_SRReferralInStatusDetails
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
 Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
 Cast(left(DateEvent,19) as datetime) as DateEvent,
 IDProfileEnteredBy,
IDDoneBy,
TextualEventDoneBy,
IDOrganisationDoneAt,
StatusOfReferralIN,
IDReferralIn,
IDEvent,
IDPatient,
sv.Digest,
IDOrganisation,
IDOrganisationRegisteredAt,
DataDeleted,
LoadNum,
deleteflag,
LoadedToFDM ,
cast(cb_rowid as int64) 
From ', sourcedb,'.tbl_SRReferralInStatusDetails sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;





#tbl_SRReferralOut
delete from   ', sourcedb,'.tbl_SRReferralOut where Digest is null ;

insert into  ',targetdb,'.tbl_SRReferralOut
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
 Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
 Cast(left(DateEvent,19) as datetime) as DateEvent,
 IDProfileEnteredBy,
IDDoneBy,
TextualEventDoneBy,
IDOrganisationDoneAt,
TypeOfReferral,
Reason,
IDProfileReferrer,
ServiceOffered,
ReReferral,
Urgency,
PrimaryDiagnosis,
RecipientID,
RecipientIDType,
IDEvent,
IDPatient,
sv.Digest,
IDOrganisation,
M101140ReasonForOutOfAreaReferralAdultAcuteMentalHealth,
IDOrganisationRegisteredAt,
DataDeleted,
LoadNum,
deleteflag,
LoadedToFDM ,
cast(cb_rowid as int64) 
From ', sourcedb,'.tbl_SRReferralOut sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;

#tbl_SRReferralOutStatusDetails
delete from   ', sourcedb,'.tbl_SRReferralOutStatusDetails where Digest is null ;

insert into  ',targetdb,'.tbl_SRReferralOutStatusDetails
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
 Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
 Cast(left(DateEvent,19) as datetime) as DateEvent,
IDOrganisationDoneAt,
IDProfileEnteredBy,
TextualEventDoneBy,
IDDoneBy,
StatusOfReferralOut,
IDReferralOut,
IDEvent,
IDPatient,
sv.Digest,
IDOrganisation,
IDOrganisationRegisteredAt,
DataDeleted,
LoadNum,
deleteflag,
LoadedToFDM ,
cast(cb_rowid as int64) 
From ', sourcedb,'.tbl_SRReferralOutStatusDetails  sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;

#tbl_SRRepeatTemplate
delete from   ', sourcedb,'.tbl_SRRepeatTemplate where Digest is null ;

insert into  ',targetdb,'.tbl_SRRepeatTemplate
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
 Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
 Cast(left(DateEvent,19) as datetime) as DateEvent,
 IDProfileEnteredBy,
IDDoneBy,
TextualEventDoneBy,
IDOrganisationDoneAt,
IDMultiLexProduct,
IDMultiLexPack,
IDMultiLexDMD,
NameOfMedication,
DateMedicationTemplateStart,
DateMedicationTemplateEnd,
DateMedicationTemplateReview,
MedicationDosage,
MedicationQuantity,
MaxIssues,
CourseLengthPerIssue,
DrugStatus,
IDReferralIN,
IDEvent,
IDPatient,
sv.Digest,
IDOrganisation,
IDOrganisationRegisteredAt,
DataDeleted,
LoadNum,
deleteflag,
LoadedToFDM ,
cb_rowid  
From ', sourcedb,'.tbl_SRRepeatTemplate  sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;

#tbl_SRRota
delete from   ', sourcedb,'.tbl_SRRota where Name is null ;

insert into  ',targetdb,'.tbl_SRRota
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
IDOrganisationVisibleTo,
 DateCreation,
IDProfileCreatedBy,
Name,
RotaType,
Location,
Code,
IDProfileOwner,
AllowOverbooking,
BookingContactNumber,
IDAppointmentRoom,
IDBranch,
DataDeleted,
LoadNum,
deleteflag,
LoadedToFDM ,
cb_rowid 
From ', sourcedb,'.tbl_SRRota;

#tbl_SRRotaSlot
delete from   ', sourcedb,'.tbl_SRRotaSlot where IDRota is null ;

insert into  ',targetdb,'.tbl_SRRotaSlot
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier,
IDOrganisationVisibleTo, 
IDRota, 
RotaSlotType, 
Duration, 
Quantity, 
EmbargoDuration, 
EmbargoExpiryTime, 
BlockedSlot, 
BookableCAnB, 
OldRowIdentifier, 
DataDeleted, 
LoadNum, 
deleteflag, 
LoadedToFDM ,
cast (cb_rowid as int64) 
From ', sourcedb,'.tbl_SRRotaSlot ;

#tbl_SRSmsConsent

delete from   ', sourcedb,'.tbl_SRSmsConsent where Digest is null ;

insert into  ',targetdb,'.tbl_SRSmsConsent
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
 Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
 Cast(left(DateEvent,19) as datetime) as DateEvent,
 IDProfileEnteredBy,
IDDoneBy,
TextualEventDoneBy,
IDOrganisationDoneAt,
SmsConsent,
IDEvent,
IDPatient,
IDOrganisation,
IDOrganisationRegisteredAt,
sv.digest,
DataDeleted,
LoadNum,
deleteflag,
LoadedToFDM ,
cast(cb_rowid as int64) 
From ', sourcedb,'.tbl_SRSmsConsent sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;


#tbl_SRStaffMember
delete from   ', sourcedb,'.tbl_SRStaffMember where StaffName is null ;

insert into  ',targetdb,'.tbl_SRStaffMember
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
StaffName,
StaffUserName,
NationalIdType,
IDNational,
IDSmartCard,
Obsolete,
DataDeleted,
LoadNum,
deleteflag,
LoadedToFDM ,
cast(cb_rowid as int64) 
From ', sourcedb,'.tbl_SRStaffMember ;
#tbl_SRStaffMemberProfile
delete from   ', sourcedb,'.tbl_SRStaffMemberProfilewhere StaffRole is null ;

insert into  ',targetdb,'.tbl_SRStaffMemberProfile
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 DateProfileCreated, 
IDProfileCreatedBy, 
IDStaffMemberProfileRole, 
StaffRole, 
DateEmploymentStart, 
DateEmploymentEnd, 
PPAID, 
GPLocalCode, 
IDStaffMember, 
IDOrganisation, 
GmpID, 
DataDeleted, 
LoadNum, 
deleteflag, 
LoadedToFDM ,
cast(cb_rowid as int64) 
From ', sourcedb,'.tbl_SRStaffMemberProfile ;


#tbl_SRStaffMemberProfileRole 
delete from   ', sourcedb,'.tbl_SRStaffMemberProfileRolewhere RoleDescription is null ;

insert into  ',targetdb,'.tbl_SRStaffMemberProfileRole
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
RoleDescription, 
IsSpineRole,
DateLastAmended,
DateDeleted,
DataDeleted,
LoadNum,
deleteflag,
LoadedToFDM ,
cast(cb_rowid as int64) 
From ', sourcedb,'.tbl_SRStaffMemberProfileRole ;





#tbl_SRTemplate
delete from   ', sourcedb,'.tbl_SRTemplatewhere TemplateName is null ;

insert into  ',targetdb,'.tbl_SRTemplate
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
IDOrganisationVisibleTo,
DatePublished,
IDProfilePublishedBy,
TemplateName,
TemplateVersion,
IDOrganisation,
DataDeleted,
LoadNum,
deleteflag,
LoadedToFDM ,
cast(cb_rowid as int64) 
From ', sourcedb,'.tbl_SRTemplate;





#tbl_SRTrust
delete from   ', sourcedb,'.tbl_SRTrustwhere Name is null ;

insert into  ',targetdb,'.tbl_SRTrust
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
Cast(left(DateCreated,19) as datetime) as DateCreated ,
IdProfileCreatedBy,
Name,
OdsCode,
HouseName,
HouseNumber,
NameOfRoad,
NameofLocality,
NameOfTown,
NameOfCounty,
FullPostCode,
Telephone,
SecondaryTelephone,
Fax,
DataDeleted,
LoadNum,
deleteflag,
LoadedToFDM ,
cast(cb_rowid as int64) 
From ', sourcedb,'.tbl_SRTrust;








#tbl_SRVariableDoseCDMedication
delete from   ', sourcedb,'.tbl_SRVariableDoseCDMedication where Digest is null ;

insert into  ',targetdb,'.tbl_SRVariableDoseCDMedication
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo,
 Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded,
 Cast(left(DateEvent,19) as datetime) as DateEvent,
IDProfileEnteredBy,
IDDoneBy,
TextualEventDoneBy,
IDOrganisationDoneAt,
IDMultiLexProduct,
IDMultiLexPack,
IDMultiLexDMD,
NameOfMedication,
DateMedicationStart,
DateMedicationEnd,
MedicationCourseLength,
SupervisedDose,
MedicationEndReason,
IDProfileMedicationEndedBy,
IDReferralIn,
IDEvent,
IDPatient,
sv.Digest,
IDOrganisation,
IDOrganisationRegisteredAt,
DataDeleted,
LoadNum,
deleteflag,
LoadedToFDM ,
cast(cb_rowid as int64) 
From ', sourcedb,'.tbl_SRVariableDoseCDMedication sv
  left join ', demo, '  demo
on sv.Digest = demo.digest;


delete From ', sourcedb,'.tbl_SRVisit 
where Digest is null ;

#tbl_SRVisit
insert into  ',targetdb,'.tbl_SRVisit
SELECT distinct cast(RowIdentifier as int64) as RowIdentifier ,
 demo.person_id ,
 IDOrganisationVisibleTo
, Cast(left(DateEventRecorded,19) as datetime) as DateEventRecorded
, IDProfileEnteredBy
, cast(left(DateBooked,19) as datetime) as DateBooked
, cast(left(DateRequested,19) as datetime) as DateRequested 
, CurrentStatus
, IDProfileRequested
, IDProfileAssigned
, FollowUpDetails
, IDReferralIN
, IDPatient
, sv.Digest
, IDOrganisation
, Duration
, IDOrganisationRegisteredAt
, DataDeleted
, LoadNum
, deleteflag
, LoadedToFDM ,
cast(cb_rowid as int64) 
From ', sourcedb,'.tbl_SRVisit sv
  left join ', demo, '  demo
on sv.Digest = demo.digest


